In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [11]:
df = pd.read_csv("drug_200.csv")

df['Sex'] = df['Sex'].map({'F': 0, 'M': 1})
df['Cholesterol'] = df['Cholesterol'].map({'NORMAL': 0, 'HIGH': 1})

bp_ohe = OneHotEncoder(sparse_output=False, drop=None)
bp_encoded = bp_ohe.fit_transform(df[['BP']])
bp_feature_names = bp_ohe.get_feature_names_out(['BP'])
bp_df = pd.DataFrame(bp_encoded, columns=bp_feature_names, index=df.index)

num_df = df[['Age', 'Na_to_K']].copy()

scaler = StandardScaler()
num_scaled = scaler.fit_transform(num_df)

X = np.hstack([num_scaled, df[['Sex', 'Cholesterol']].values, bp_df.values])

X = np.hstack([np.ones((X.shape[0], 1)), X])

le = LabelEncoder()
y = le.fit_transform(df['Drug'].values)
n_classes = len(np.unique(y))

Y_onehot = np.eye(n_classes)[y]

In [12]:
def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    expz = np.exp(z)
    return expz / np.sum(expz, axis=1, keepdims=True)

def compute_loss(X, Y_onehot, W, reg_type=None, lam=0.0, alpha=0.5):
    N = X.shape[0]
    logits = X @ W
    probs = softmax(logits)
    # negative log-likelihood / cross-entropy
    loss = -np.sum(Y_onehot * np.log(probs + 1e-15)) / N

    # exclude bias (first row of W corresponds to bias weights for each class)
    W_no_bias = W.copy()
    W_no_bias[0, :] = 0.0

    if reg_type == "l1":
        loss += lam * np.sum(np.abs(W_no_bias))
    elif reg_type == "l2":
        loss += lam * np.sum(W_no_bias ** 2)
    elif reg_type == "elasticnet":
        loss += lam * (alpha * np.sum(np.abs(W_no_bias)) + (1 - alpha) * np.sum(W_no_bias ** 2))
    return loss

def compute_gradient(X, Y_onehot, W, reg_type=None, lam=0.0, alpha=0.5):
    N = X.shape[0]
    logits = X @ W
    probs = softmax(logits)
    grad = (X.T @ (probs - Y_onehot)) / N  # shape (n_features, n_classes)

    # Regularize but exclude bias row (row 0)
    if reg_type is not None and lam > 0:
        reg_term = np.zeros_like(W)
        if reg_type == "l1":
            reg_term = lam * np.sign(W)
        elif reg_type == "l2":
            reg_term = 2 * lam * W
        elif reg_type == "elasticnet":
            reg_term = lam * (alpha * np.sign(W) + 2 * (1 - alpha) * W)

        reg_term[0, :] = 0.0  # exclude bias from regularization
        grad += reg_term

    return grad


In [13]:
def train_softmax(X_train, Y_train_onehot, lr=0.05, epochs=1500, reg_type=None, lam=0.0, alpha=0.5, verbose=False):
    n_features = X_train.shape[1]
    n_classes = Y_train_onehot.shape[1]
    W = np.zeros((n_features, n_classes))
    for epoch in range(epochs):
        grad = compute_gradient(X_train, Y_train_onehot, W, reg_type, lam, alpha)
        W -= lr * grad
        if verbose and (epoch % 300 == 0 or epoch == epochs-1):
            loss = compute_loss(X_train, Y_train_onehot, W, reg_type, lam, alpha)
            print(f" epoch {epoch:4d}  loss={loss:.6f}")
    return W

def predict(X, W):
    probs = softmax(X @ W)
    return np.argmax(probs, axis=1)

In [14]:
def cross_validate(X, y, Y_onehot, reg_type=None, lam=0.0, alpha=0.5, lr=0.05, epochs=1500):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {"accuracy": [], "precision": [], "recall": [], "f1": []}

    for train_idx, test_idx in kf.split(X):
        X_tr, X_te = X[train_idx], X[test_idx]
        Ytr_onehot, y_te = Y_onehot[train_idx], y[test_idx]

        W = train_softmax(X_tr, Ytr_onehot, lr=lr, epochs=epochs,
                          reg_type=reg_type, lam=lam, alpha=alpha, verbose=False)
        y_pred = predict(X_te, W)
        
        print(W)


        metrics["accuracy"].append(accuracy_score(y_te, y_pred))
        metrics["precision"].append(precision_score(y_te, y_pred, average='macro', zero_division=0))
        metrics["recall"].append(recall_score(y_te, y_pred, average='macro', zero_division=0))
        metrics["f1"].append(f1_score(y_te, y_pred, average='macro', zero_division=0))
    return {k: np.mean(v) for k, v in metrics.items()}

In [15]:
print("Training & evaluating...")

base = cross_validate(X, y, Y_onehot, reg_type=None, lam=0.0)
lasso = cross_validate(X, y, Y_onehot, reg_type="l1", lam=0.005)
ridge = cross_validate(X, y, Y_onehot, reg_type="l2", lam=0.005)
elastic = cross_validate(X, y, Y_onehot, reg_type="elasticnet", lam=0.005, alpha=0.5)

results_df = pd.DataFrame({
    "Model": ["No Reg", "Lasso (L1)", "Ridge (L2)", "ElasticNet"],
    "Accuracy": [base["accuracy"], lasso["accuracy"], ridge["accuracy"], elastic["accuracy"]],
    "Precision": [base["precision"], lasso["precision"], ridge["precision"], elastic["precision"]],
    "Recall": [base["recall"], lasso["recall"], ridge["recall"], elastic["recall"]],
    "F1-score": [base["f1"], lasso["f1"], ridge["f1"], elastic["f1"]],
})

print("\n5-Fold CV results (macro averaged):")
print(results_df.round(4))

Training & evaluating...
[[-0.65911165 -0.67691902 -0.92023057  0.65047115  1.6057901 ]
 [-0.86670139  1.3669853  -0.25736227 -0.19889935 -0.04402229]
 [-1.18156746 -0.4372645  -0.73676928 -1.80044342  4.15604467]
 [ 0.09231377 -0.27770134  0.04343246 -0.33190027  0.47385539]
 [ 0.02284131 -0.24987923  1.08519304 -1.0504275   0.19227237]
 [ 1.38700271  1.24571076 -1.17914495 -1.90289668  0.44932815]
 [-1.06938699 -1.03455929  1.21654998  0.47368068  0.41371561]
 [-0.97672738 -0.88807049 -0.95763561  2.07968715  0.74274633]]
[[-0.67524662 -0.69125332 -0.95372406  0.7774934   1.54273061]
 [-0.74098631  1.54563644 -0.40827824 -0.19596841 -0.20040347]
 [-1.13384873 -0.35735926 -0.96450015 -1.66176317  4.11747131]
 [ 0.07213848 -0.08501171 -0.22643748 -0.00545083  0.24476154]
 [ 0.29986804 -0.48331554  1.34051655 -1.44141487  0.28434582]
 [ 1.46434689  1.31293    -1.30027541 -1.93702501  0.46002353]
 [-1.18986817 -1.22891033  1.36190958  0.60301401  0.4538549 ]
 [-0.94972534 -0.775273   -1.